# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-3568e26c20-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Disney explorative search") 

Consider the following exploratory information need:

> investigate films produced by Disney and compare workers among films.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36479`    | The Lion King      | node |





Also consider

```
wd:Q36479 ?p ?obj .
```

is the BGP to retrieve all **properties of The Lion King**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for Walt Disney Animation Studios

3. Identify the BGP for Walt Disney (person)

4. List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

5. Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

6. Identify the BGP for the film "The Little Mermaid"

7. Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label). 

8. Identify the BGP for Academy Awards

9. For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [6]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [6]:
# aim: check if the film 'The Lion King' film has the property 'instanceOf' that might relate it to a possible film class
# result: the property 'instanceOf' has been found
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [10]:
# aim: find the class to which the film 'Lion King' belongs
# result: the class found is not 'film' but 'animated feature film'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # set the film 'The Lion King' and the property 'instanceOf' to get the related object(s)
   wd:Q36479 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q29168811'), ('name', 'animated feature film')]
1


In [11]:
# aim: find the properties of the 'animated feature film' class to see possible links to a 'film' class
# result: the property 'subclass of' has been found
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q29168811 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
4


In [14]:
# aim: find what is the super class of 'animated feature film'
# result: the classes 'animated film' and 'feature film' have been found
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q29168811 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q202866'), ('name', 'animated film')]
[('obj', 'http://www.wikidata.org/entity/Q24869'), ('name', 'feature film')]
2


Final query for this task

In [15]:
# aim: find what are the super classes of 'animated film' or 'feature film'
# result: the class 'film' has been found!
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?filmClass wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER(?filmClass IN (wd:Q202866, wd:Q24869))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


## Task 2
Identify the BGP for Walt Disney Animation Studios

In [19]:
# aim: find the 'producer' of the film 'Lion King'
# result: we found a person instead of the company; we look better to the properties of 'Lion King' and we find 'production company' that may suits
#         better for the next query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # set the film 'The Lion King' and the property 'producer' to get the related object(s)
   wd:Q36479 wdt:P162 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1072843'), ('name', 'Don Hahn')]
1


Final query for this task

In [20]:
# aim: find the 'production company' of the film 'Lion King'
# result: we found the 'Walt Disney Animation Studios'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # set the film 'Lion King' and the property 'production company' to get the related object(s)
   wd:Q36479 wdt:P272 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q191224'), ('name', 'Walt Disney Pictures')]
[('obj', 'http://www.wikidata.org/entity/Q1047410'), ('name', 'Walt Disney Animation Studios')]
2


## Task 3
Identify the BGP for Walt Disney (person)

In [12]:
# aim: find the properties of the company 'Walt Disney Animation Studios' 
# result: we found the property 'founded by'
queryString = """
SELECT DISTINCT ?s ?name ?p ?pname
WHERE {
   # set the company 'Walt Disney Animation Studios' and retrieve all the properties
    ?s ?p wd:Q1047410 .
   # get the label
   ?s sc:name ?name.
   ?p sc:name ?pname.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q2900817'), ('name', 'Bianca Majolie'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q2396556'), ('name', 'Mark Henn'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q2525726'), ('name', 'Bill Tytla'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q27865787'), ('name', 'Broose Johnson'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q27913315'), ('name', 'James Young Jackson'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q27915447'), ('name', 'Theodore Ty'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q27929425'), ('name', 'Tony Stanley'), 

In [5]:
# aim: find the properties of the company 'Walt Disney Animation Studios' 
# result: we found the property 'founded by'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # set the company 'Walt Disney Animation Studios' and retrieve all the properties
   wd:Q1047410 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('name', 'director / manager')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/

Final query for this task

In [25]:
# aim: find the object related to the company 'Walt Disney Animation Studios' through the property 'founded by'
# result: we found the person 'Walt Disney'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # set the company 'Walt Disney Animation Studios' and the property 'founded by' to retrieve all the objects
   wd:Q1047410 wdt:P112 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8704'), ('name', 'Walt Disney')]
[('obj', 'http://www.wikidata.org/entity/Q347432'), ('name', 'Roy O. Disney')]
2


## Task 4
List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

In [6]:
# aim: find the properties of the person 'Walt Disney' 
# result: we didn't find good properties but the LIMIT can be increased in the next query to find more properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # set the person 'Walt Disney' to retrieve the properties
   wd:Q8704 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5622'), ('name', 'National Cartoonists Society member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata

In [3]:
# aim: find the properties of the person 'Walt Disney' 
# result: we found the properties 'date of birth' and 'date of death'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # set the person 'Walt Disney' to retrieve the properties
   wd:Q8704 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5622'), ('name', 'National Cartoonists Society member ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata

In [13]:
# aim: find the values of 'date of birth' and 'date of death' of the person 'Walt Disney' 
# result: we found the range of time in which 'Walt Disney' was alive
queryString = """
SELECT DISTINCT ?name ?obj
WHERE {
   # set the person 'Walt Disney' to retrieve the value of the properties 'date of birth' and 'date of death'
   wd:Q8704 ?p ?obj .
   # get the label
   ?p sc:name ?name .
   FILTER(?p IN (wdt:P569, wdt:P570))
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'date of birth'), ('obj', '1901-12-05T00:00:00Z')]
[('name', 'date of death'), ('obj', '1966-12-15T00:00:00Z')]
2


In [5]:
# aim: find all the films produced by the 'Walt Disney Animation Studios'
# result: we found the films produced by 'Walt Disney Animation Studios'
queryString = """
SELECT DISTINCT ?film ?name
WHERE {
   # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
   ?film wdt:P272 wd:Q1047410 .
   # get the label
   ?film sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove")]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('name', 'Beauty and the Beast')]
[('film', 'http://www.wikidata.org/entity/Q15046569'), ('name', 'Roger Rabbit short films')]
[('film', 'http://www.wikidata.org/entity/Q3362144'), ('name', 'Trail Mix-Up')]
[('film', 'http://www.wikidata.org/entity/Q379873'), ('name', 'Who Framed Roger Rabbit')]
[('film', 'http://www.wikidata.org/entity/Q36479'), ('name', 'The Lion King')]
[('film', 'http://www.wikidata.org/entity/Q215617'), ('name', 'Sleeping Beauty')]
[('film', 'http://www.wikidata.org/entity/Q212792'), ('name', 'Bolt')]
[('film', 'http://www.wikidata.org/entity/Q944287'), ('name', 'The Little Mermaid')]
[('film', 'http://www.wikidata.org/entity/Q3236849'), ('name', "Let's Stick Together")]
[('film', 'http://www.wikidata.org/entity/Q25167044'), ('name', 'Ralph B

In [2]:
# aim: check once again the properties for the film 'Lion King' by increasing the LIMIT to 200 (instead of 100 in the first query)
#      to see if there are other properties apart from 'publication date' that might refer to the date the film was created     
# result: no other properties seem to refer to a creation date

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [3]:
# aim: check the property 'publication date' for the film 'Lion King' to see if it's actually a date. 
# result: yes, this could be the date the film was created

queryString = """
SELECT DISTINCT ?obj
WHERE {
   # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
   wd:Q36479 wdt:P577 ?obj .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '1994-06-15T00:00:00Z')]
1


In [4]:
# aim: get all the films produced by 'Walt Disney Animation Studios'
# result: some films have more than one publication date so we try to investigate if other properties exist

queryString = """
SELECT DISTINCT ?film ?name ?filmdate
WHERE {
   # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
   ?film wdt:P272 wd:Q1047410 ;
         # get the label
         sc:name ?name ;
         # get the value of 'publication date'
         wdt:P577 ?filmdate .
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove"), ('filmdate', '2000-12-15T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove"), ('filmdate', '2001-02-09T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove"), ('filmdate', '2001-03-15T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-11-23T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-11-30T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-12-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-12-02T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-12-22T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q17

In [5]:
# aim: check the properties for the film 'The Emperor's New Groove' to see if there are other properties apart from 'publication date' that might refer to the date the film was created     
# result: no other properties seem to refer to a creation date... We decide to pick only the minimum date in case of duplicates for the next queries

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q223163 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('name', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1434'), ('name', 'takes place in fictional universe')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', '

In [6]:
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their 'publication date'
# result: all the films produced by the 'Walt Disney Animation Studios' with their 'publication date' considering its minimum value if more than one is associated 

queryString = """
SELECT DISTINCT ?film ?name (MIN(?filmdate) AS ?datecreation)
WHERE {
   ?film wdt:P179 wd:Q56070713 ;
       sc:name ?name;
       wdt:P577 ?filmdate .
} GROUP BY ?film ?name
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q943192'), ('name', 'Fantasia'), ('datecreation', '1940-11-13T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q40302'), ('name', 'The Fox and the Hound'), ('datecreation', '1981-07-10T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q246283'), ('name', 'Frozen'), ('datecreation', '2013-11-10T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q134430'), ('name', 'Snow White and the Seven Dwarfs'), ('datecreation', '1937-12-21T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q736731'), ('name', 'The Three Caballeros'), ('datecreation', '1944-12-21T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q4341553'), ('name', 'Pinocchio'), ('datecreation', '1940-02-23T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q172284'), ('name', 'Lady and the Tramp'), ('datecreation', '1955-06-16T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q36479'), ('name', 'The Lion King'), ('datecreation', '1994-06-15T00:00:00

In [7]:
# (The query has been rewritten a few times because of the problem of comparing dates by means of the xsd prefix)
# aim: find all the films produced by the 'Walt Disney Animation Studios' while 'Walt Disney' was alive
# result: all the films produced by the 'Walt Disney Animation Studios' with their 'publication date' considering its minimum value if more than one is associated. The
#         films are all publicated while 'Walt Disney' was alive

queryString = """
SELECT DISTINCT ?film ?name WHERE{

   {
      SELECT DISTINCT ?film ?name (MIN(?filmdate) AS ?datecreation)
      WHERE {
         # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
         ?film wdt:P272 wd:Q1047410 ;
               # get the label
               sc:name ?name ;
               # get the value of 'publication date'
               wdt:P577 ?filmdate .
      }GROUP BY ?film ?name
       LIMIT 200
 
   }
   FILTER (?datecreation >= "1901-12-05T00:00:00Z"^^xsd:dateTime && ?datecreation <= "1966-12-15T00:00:00Z"^^xsd:dateTime)
}
LIMIT 200
"""
print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3392773'), ('name', 'Playful Pluto')]
[('film', 'http://www.wikidata.org/entity/Q3028329'), ('name', 'Two Weeks Vacation')]
[('film', 'http://www.wikidata.org/entity/Q3228508'), ('name', 'Hold That Pose')]
[('film', 'http://www.wikidata.org/entity/Q3178943'), ('name', "Donald's Dilemma")]
[('film', 'http://www.wikidata.org/entity/Q3006129'), ('name', 'Bearly Asleep')]
[('film', 'http://www.wikidata.org/entity/Q2399982'), ('name', 'Tea for Two Hundred')]
[('film', 'http://www.wikidata.org/entity/Q215617'), ('name', 'Sleeping Beauty')]
[('film', 'http://www.wikidata.org/entity/Q3036139'), ('name', 'Beezy Bear')]
[('film', 'http://www.wikidata.org/entity/Q3520106'), ('name', 'The Brave Engineer')]
[('film', 'http://www.wikidata.org/entity/Q2090328'), ('name', 'Trick or Treat')]
[('film', 'http://www.wikidata.org/entity/Q640679'), ('name', 'Rugged Bear')]
[('film', 'http://www.wikidata.org/entity/Q3207087'), ('name', 'Hockey Homicide')]
[(

In [15]:
# After reviewing the solutions for this task we noticed 'Mickey Mouse Mixed-Up Adventures' which could be something else than a film... Therefore we tried applying queries once again to
# check if the result set found is okay or not
# aim: get if 'Mickey Mouse Mixed-Up Adventures' is a film or not since the name reminds some kind of cartoon series
# result: yes, it is an 'instance of' television series so in the precedent list there can be resources that are not actually films so we decide to inspect them all in the next query
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oname
WHERE {
   # set the person 'Walt Disney' to retrieve the value of the properties 'date of birth' and 'date of death'
   wd:Q24970452 ?p ?obj .
   # get the label
   ?p sc:name ?name .
   ?obj sc:name ?oname
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P272'), ('name', 'production company'), ('obj', 'http://www.wikidata.org/entity/Q1392453'), ('oname', 'Disney Television Animation')]
[('p', 'http://www.wikidata.org/prop/direct/P725'), ('name', 'voice actor'), ('obj', 'http://www.wikidata.org/entity/Q1364909'), ('oname', 'Bill Farmer')]
[('p', 'http://www.wikidata.org/prop/direct/P725'), ('name', 'voice actor'), ('obj', 'http://www.wikidata.org/entity/Q837676'), ('oname', 'Dee Bradley Baker')]
[('p', 'http://www.wikidata.org/prop/direct/P725'), ('name', 'voice actor'), ('obj', 'http://www.wikidata.org/entity/Q311987'), ('oname', 'Rob Paulsen')]
[('p', 'http://www.wikidata.org/prop/direct/P725'), ('name', 'voice actor'), ('obj', 'http://www.wikidata.org/entity/Q1075796'), ('oname', 'Carlos Alazraqui')]
[('p', 'http://www.wikidata.org/prop/direct/P7501'), ('name', 'audio system'), ('obj', 'http://www.wikidata.org/entity/Q34678'), ('oname', 'stereo')]
[('p', 'http://www.wikidata.org/pro

In [16]:
# aim: find all the resources produced by the 'Walt Disney Animation Studios'
# result: we found the resources produced by 'Walt Disney Animation Studios'. Here we find also 'The Little Mermaid' that is asked as one of the next BGP to be found but we notice that this is not
#         the movie but a tv series. Therefore we try to inspect it in the next query since the movie 'The Little Mermaid' should have been made by Disney but it's not in the list. This probably
#         means that there should be another way to find other films connected to 'Walt Disney Animation Studios'
queryString = """
SELECT DISTINCT ?film ?name ?instanceof ?instanceofname
WHERE {
   # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
   ?film wdt:P272 wd:Q1047410 ;
       wdt:P31 ?instanceof .
   # get the label
   ?film sc:name ?name.
   ?instanceof sc:name ?instanceofname
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3362144'), ('name', 'Trail Mix-Up'), ('instanceof', 'http://www.wikidata.org/entity/Q17517379'), ('instanceofname', 'animated short film')]
[('film', 'http://www.wikidata.org/entity/Q3236849'), ('name', "Let's Stick Together"), ('instanceof', 'http://www.wikidata.org/entity/Q17517379'), ('instanceofname', 'animated short film')]
[('film', 'http://www.wikidata.org/entity/Q18398561'), ('name', 'Frozen Fever'), ('instanceof', 'http://www.wikidata.org/entity/Q17517379'), ('instanceofname', 'animated short film')]
[('film', 'http://www.wikidata.org/entity/Q2090328'), ('name', 'Trick or Treat'), ('instanceof', 'http://www.wikidata.org/entity/Q17517379'), ('instanceofname', 'animated short film')]
[('film', 'http://www.wikidata.org/entity/Q640679'), ('name', 'Rugged Bear'), ('instanceof', 'http://www.wikidata.org/entity/Q17517379'), ('instanceofname', 'animated short film')]
[('film', 'http://www.wikidata.org/entity/Q2252695'), ('name', 'Pape

In [18]:
# aim: get all the properties of 'The Little Mermaid' tv series to find if we can connect it to the original film
# result: we found the 'based on'
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   wd:Q944287 ?p ?o .
   ?p sc:name ?pname.
    
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('pname', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('pname', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('pname', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('pname', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('pname', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('pname', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2364'), ('pname', 'production code')]
[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('pname', 'number of seasons')]
[('p', 'http://www.wikidata.org/prop/direct/P2603'), ('pname', 'Kinopoisk film ID')]
[('p', 'http://www.wikidata.org/pr

In [19]:
# aim: get the film 'The Little Mermaid' to see if it connects to Disney in some other ways to find other films possibly connected to Disney
# result: we found the film 'The Little Mermaid'
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   wd:Q944287 wdt:P144 ?o .
   ?o sc:name ?oname.
    
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q190135'), ('oname', 'The Little Mermaid')]
1


In [21]:
# aim: get all the properties and objects of the film 'The Little Mermaid' to get if we can find it is related to Disney
# result: we found many relationships with resources such as - 'production company':'Walt Disney Pictures'
#                                                            - 'distributed by':'Walt Disney Studios Motion Pictures'
#                                                            - 'part of the series': 'Walt Disney Studios films' ...
#         This should mean that it is related to Walt Disney Studios therefore we try to investigate the property 'part of the series' since it could be a list of films created by 'Walt Disney Studios'

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   wd:Q190135 ?p ?o .
   ?p sc:name ?pname.
   ?o sc:name ?oname
    
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P725'), ('pname', 'voice actor'), ('o', 'http://www.wikidata.org/entity/Q1341627'), ('oname', 'Rod McKuen')]
[('p', 'http://www.wikidata.org/prop/direct/P6942'), ('pname', 'animator'), ('o', 'http://www.wikidata.org/entity/Q2396556'), ('oname', 'Mark Henn')]
[('p', 'http://www.wikidata.org/prop/direct/P6942'), ('pname', 'animator'), ('o', 'http://www.wikidata.org/entity/Q11853913'), ('oname', 'Barry Cook')]
[('p', 'http://www.wikidata.org/prop/direct/P6942'), ('pname', 'animator'), ('o', 'http://www.wikidata.org/entity/Q12320808'), ('oname', 'Jorgen Klubien')]
[('p', 'http://www.wikidata.org/prop/direct/P6942'), ('pname', 'animator'), ('o', 'http://www.wikidata.org/entity/Q28790229'), ('oname', 'Phillip Young')]
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('pname', 'film crew member'), ('o', 'http://www.wikidata.org/entity/Q28803503'), ('oname', 'Robert Stanton')]
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('pname',

In [22]:
# aim: get all the subjects connected through 'part of the series' to 'Walt Disney Animation Studios film' to get all the films related to it
# result: we got all the films produced by 'Walt Disney Animation Studios'
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
   ?s wdt:P179 wd:Q56070713 .
   ?s sc:name ?sname.
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q223163'), ('sname', "The Emperor's New Groove")]
[('s', 'http://www.wikidata.org/entity/Q134430'), ('sname', 'Snow White and the Seven Dwarfs')]
[('s', 'http://www.wikidata.org/entity/Q18647981'), ('sname', 'Moana')]
[('s', 'http://www.wikidata.org/entity/Q199839'), ('sname', 'The Jungle Book')]
[('s', 'http://www.wikidata.org/entity/Q216929'), ('sname', 'Oliver & Company')]
[('s', 'http://www.wikidata.org/entity/Q329316'), ('sname', 'The Black Cauldron')]
[('s', 'http://www.wikidata.org/entity/Q537407'), ('sname', 'Mulan')]
[('s', 'http://www.wikidata.org/entity/Q270470'), ('sname', 'Peter Pan')]
[('s', 'http://www.wikidata.org/entity/Q179673'), ('sname', 'Beauty and the Beast')]
[('s', 'http://www.wikidata.org/entity/Q215365'), ('sname', 'Brother Bear')]
[('s', 'http://www.wikidata.org/entity/Q936194'), ('sname', 'Home on the Range')]
[('s', 'http://www.wikidata.org/entity/Q36479'), ('sname', 'The Lion King')]
[('s', 'http://www.wikidat

In [27]:
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their 'publication date'
# result: we found the films produced by the company 'Walt Disney Animation Studios' with their dates but some films have more than one publication date

queryString = """
SELECT DISTINCT ?film ?name ?filmdate
WHERE {
    {
        SELECT DISTINCT ?film ?name ?filmdate WHERE{
            ?film wdt:P179 wd:Q56070713 ;
               sc:name ?name;
               wdt:P577 ?filmdate .
        }
    }
    UNION
    {
        # the films having as 'production company' 'Walt Disney Animation Studios'
        SELECT DISTINCT ?film ?name ?filmdate WHERE {
            # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
            ?film wdt:P272 wd:Q1047410 ;
                # get the label
                sc:name ?name ;
                # get the type of film
                wdt:P31 ?instanceof ;
                # get the value of 'publication date'
                wdt:P577 ?filmdate .
            # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
            # and 'Walt Disney Animation Studios film'
            FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
        }
    }
}
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove"), ('filmdate', '2000-12-15T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove"), ('filmdate', '2001-02-09T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove"), ('filmdate', '2001-03-15T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q134430'), ('name', 'Snow White and the Seven Dwarfs'), ('filmdate', '1937-12-21T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-11-23T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-11-30T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-12-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('filmdate', '2016-12-02T00:00:00Z')]
[('film', 'http://www

In [28]:
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their 'publication date'
# result: all the films produced by the 'Walt Disney Animation Studios' with their 'publication date' considering its minimum value if more than one is associated 

queryString = """
SELECT DISTINCT ?film ?name (MIN(?filmdate) AS ?datecreation)
WHERE {
    {
        SELECT DISTINCT ?film ?name ?filmdate WHERE{
            ?film wdt:P179 wd:Q56070713 ;
               sc:name ?name;
               wdt:P577 ?filmdate .
        }
    }
    UNION
    {
        # the films having as 'production company' 'Walt Disney Animation Studios'
        SELECT DISTINCT ?film ?name ?filmdate WHERE {
            # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
            ?film wdt:P272 wd:Q1047410 ;
                # get the label
                sc:name ?name ;
                # get the type of film
                wdt:P31 ?instanceof ;
                # get the value of 'publication date'
                wdt:P577 ?filmdate .
            # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
            # and 'Walt Disney Animation Studios film'
            FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
        }
    }
} GROUP BY ?film ?name
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q943192'), ('name', 'Fantasia'), ('datecreation', '1940-11-13T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q40302'), ('name', 'The Fox and the Hound'), ('datecreation', '1981-07-10T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q246283'), ('name', 'Frozen'), ('datecreation', '2013-11-10T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q134430'), ('name', 'Snow White and the Seven Dwarfs'), ('datecreation', '1937-12-21T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q736731'), ('name', 'The Three Caballeros'), ('datecreation', '1944-12-21T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q869993'), ('name', 'A Goofy Movie'), ('datecreation', '1995-04-07T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q5282372'), ('name', "Disney's American Legends"), ('datecreation', '2002-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q4341553'), ('name', 'Pinocchio'), ('datecreation', '1940-02-23

Final query for this task

In [29]:
# aim: find all the films produced by the 'Walt Disney Animation Studios' while 'Walt Disney' was alive
# result: all the films produced by the 'Walt Disney Animation Studios' with their 'publication date' considering its minimum value if more than one is associated. The
#         films are all publicated while 'Walt Disney' was alive

queryString = """
SELECT DISTINCT ?film ?name WHERE{
   {
        SELECT DISTINCT ?film ?name (MIN(?filmdate) AS ?datecreation) WHERE {
            {
                SELECT DISTINCT ?film ?name ?filmdate WHERE{
                    ?film wdt:P179 wd:Q56070713 ;
                       sc:name ?name;
                       wdt:P577 ?filmdate .
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film ?name ?filmdate WHERE {
                    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
                    ?film wdt:P272 wd:Q1047410 ;
                        # get the label
                        sc:name ?name ;
                        # get the type of film
                        wdt:P31 ?instanceof ;
                        # get the value of 'publication date'
                        wdt:P577 ?filmdate .
                    # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
                    # and 'Walt Disney Animation Studios film'
                    FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        } GROUP BY ?film ?name
 
   }
   FILTER (?datecreation >= "1901-12-05T00:00:00Z"^^xsd:dateTime && ?datecreation <= "1966-12-15T00:00:00Z"^^xsd:dateTime)
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q943192'), ('name', 'Fantasia')]
[('film', 'http://www.wikidata.org/entity/Q134430'), ('name', 'Snow White and the Seven Dwarfs')]
[('film', 'http://www.wikidata.org/entity/Q736731'), ('name', 'The Three Caballeros')]
[('film', 'http://www.wikidata.org/entity/Q4341553'), ('name', 'Pinocchio')]
[('film', 'http://www.wikidata.org/entity/Q172284'), ('name', 'Lady and the Tramp')]
[('film', 'http://www.wikidata.org/entity/Q270470'), ('name', 'Peter Pan')]
[('film', 'http://www.wikidata.org/entity/Q43051'), ('name', 'Bambi')]
[('film', 'http://www.wikidata.org/entity/Q215617'), ('name', 'Sleeping Beauty')]
[('film', 'http://www.wikidata.org/entity/Q165512'), ('name', 'One Hundred and One Dalmatians')]
[('film', 'http://www.wikidata.org/entity/Q3520106'), ('name', 'The Brave Engineer')]
[('film', 'http://www.wikidata.org/entity/Q754736'), ('name', 'Make Mine Music')]
[('film', 'http://www.wikidata.org/entity/Q869741'), ('name', 'Melody Time')

## Task 5
Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

In [8]:
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their 'voice actor'(s)
# result: we found the films produced by the company 'Walt Disney Animation Studios' with their 'voice actor' (s)

queryString = """
SELECT DISTINCT ?film ?filmname ?voiceactor ?voiceactorname
WHERE {
    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
    ?film wdt:P272 wd:Q1047410 ;
           # get the label of the film
           sc:name ?filmname ;
           # get the 'voice actor'
           wdt:P725 ?voiceactor .
    # get the label of the voice actor       
    ?voiceactor sc:name ?voiceactorname
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q215617'), ('filmname', 'Sleeping Beauty'), ('voiceactor', 'http://www.wikidata.org/entity/Q2903350'), ('voiceactorname', 'Bill Shirley')]
[('film', 'http://www.wikidata.org/entity/Q15270647'), ('filmname', 'Zootopia'), ('voiceactor', 'http://www.wikidata.org/entity/Q24574523'), ('voiceactorname', 'Gita Reddy')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('filmname', 'Moana'), ('voiceactor', 'http://www.wikidata.org/entity/Q439315'), ('voiceactorname', 'Jemaine Clement')]
[('film', 'http://www.wikidata.org/entity/Q22773153'), ('filmname', "Milo Murphy's Law"), ('voiceactor', 'http://www.wikidata.org/entity/Q439315'), ('voiceactorname', 'Jemaine Clement')]
[('film', 'http://www.wikidata.org/entity/Q3362144'), ('filmname', 'Trail Mix-Up'), ('voiceactor', 'http://www.wikidata.org/entity/Q222008'), ('voiceactorname', 'Kathleen Turner')]
[('film', 'http://www.wikidata.org/entity/Q379873'), ('filmname', 'Who Framed Roger Rabbit'),

In [9]:
# aim: find the number of films produced by the 'Walt Disney Animation Studios' with their corresponding 'voice actor'(s)
# result: we found the number of films produced by the company 'Walt Disney Animation Studios' with their 'voice actor' (s)

queryString = """
SELECT DISTINCT ?voiceactor ?voiceactorname (COUNT(?film) AS ?numberfilms)
WHERE {
    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
    ?film wdt:P272 wd:Q1047410 ;
           # get the 'voice actor'
           wdt:P725 ?voiceactor .
    # get the label of the voice actor       
    ?voiceactor sc:name ?voiceactorname
}GROUP BY ?voiceactor ?voiceactorname
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('voiceactor', 'http://www.wikidata.org/entity/Q376981'), ('voiceactorname', 'Peter Firth'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q526620'), ('voiceactorname', 'Alfre Woodard'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q272960'), ('voiceactorname', 'Stefanie Scott'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q313266'), ('voiceactorname', 'John DiMaggio'), ('numberfilms', '3')]
[('voiceactor', 'http://www.wikidata.org/entity/Q230383'), ('voiceactorname', 'Kelly Macdonald'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q2070099'), ('voiceactorname', 'Flora Finch'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q40337'), ('voiceactorname', 'Sandra Oh'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q723021'), ('voiceactorname', 'Daran Norris'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q71426

In [10]:
# aim: find the number of films produced by the 'Walt Disney Animation Studios' with their corresponding 'voice actor'(s). Each voice actor must have at least worked in 6 films
# result: we found the number of films produced by the company 'Walt Disney Animation Studios' with their 'voice actor' (s)

queryString = """
SELECT DISTINCT ?voiceactor ?voiceactorname (COUNT(?film) AS ?numberfilms)
WHERE {
    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
    ?film wdt:P272 wd:Q1047410 ;
           # get the 'voice actor'
           wdt:P725 ?voiceactor .
    # get the label of the voice actor       
    ?voiceactor sc:name ?voiceactorname
}GROUP BY ?voiceactor ?voiceactorname
HAVING (COUNT(?film)>5)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('voiceactor', 'http://www.wikidata.org/entity/Q296577'), ('voiceactorname', 'Frank Welker'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q349350'), ('voiceactorname', 'Alan Tudyk'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q178882'), ('voiceactorname', 'Kristen Bell'), ('numberfilms', '6')]
[('voiceactor', 'http://www.wikidata.org/entity/Q310292'), ('voiceactorname', 'Jim Cummings'), ('numberfilms', '10')]
[('voiceactor', 'http://www.wikidata.org/entity/Q264418'), ('voiceactorname', 'Russi Taylor'), ('numberfilms', '6')]
5


In [11]:
# aim: find the number of films produced by the 'Walt Disney Animation Studios' with their corresponding 'voice actor'(s). Each voice actor must have at least worked in 6 films
# result: we found the number of films produced by the company 'Walt Disney Animation Studios' with their 'voice actor' (s)

queryString = """
SELECT DISTINCT ?voiceactor ?voiceactorname (COUNT(?film) AS ?numberfilms)
WHERE {
    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
    ?film wdt:P272 wd:Q1047410 ;
           # get the 'voice actor'
           wdt:P725 ?voiceactor .
    # get the label of the voice actor       
    ?voiceactor sc:name ?voiceactorname
}GROUP BY ?voiceactor ?voiceactorname
HAVING (COUNT(?film)>5)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('voiceactor', 'http://www.wikidata.org/entity/Q296577'), ('voiceactorname', 'Frank Welker'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q349350'), ('voiceactorname', 'Alan Tudyk'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q178882'), ('voiceactorname', 'Kristen Bell'), ('numberfilms', '6')]
[('voiceactor', 'http://www.wikidata.org/entity/Q310292'), ('voiceactorname', 'Jim Cummings'), ('numberfilms', '10')]
[('voiceactor', 'http://www.wikidata.org/entity/Q264418'), ('voiceactorname', 'Russi Taylor'), ('numberfilms', '6')]
5


In [38]:
# We redo task 5 because of the error that was done in task 4. In this case we consider only the films and not series or other resources
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their 'voice actor'(s)
# result: we found the films produced by the company 'Walt Disney Animation Studios' with their 'voice actor' (s)

queryString = """
SELECT DISTINCT ?film ?name ?voiceactor ?voiceactorname WHERE {
    ?film wdt:P725 ?voiceactor .
    ?voiceactor sc:name ?voiceactorname

    {
        SELECT DISTINCT ?film ?name WHERE{
            ?film wdt:P179 wd:Q56070713 ;
               sc:name ?name.
               
        }
    }
    UNION
    {
        # the films having as 'production company' 'Walt Disney Animation Studios'
        SELECT DISTINCT ?film ?name WHERE {
            # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
            ?film wdt:P272 wd:Q1047410 ;
                # get the label
                sc:name ?name ;
                # get the type of film
                wdt:P31 ?instanceof .
            # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
            # and 'Walt Disney Animation Studios film'
            FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
        }
    }
}
ORDER BY ASC (?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q869993'), ('name', 'A Goofy Movie'), ('voiceactor', 'http://www.wikidata.org/entity/Q235754'), ('voiceactorname', 'Kellie Martin')]
[('film', 'http://www.wikidata.org/entity/Q869993'), ('name', 'A Goofy Movie'), ('voiceactor', 'http://www.wikidata.org/entity/Q311068'), ('voiceactorname', 'Wallace Shawn')]
[('film', 'http://www.wikidata.org/entity/Q869993'), ('name', 'A Goofy Movie'), ('voiceactor', 'http://www.wikidata.org/entity/Q1364909'), ('voiceactorname', 'Bill Farmer')]
[('film', 'http://www.wikidata.org/entity/Q869993'), ('name', 'A Goofy Movie'), ('voiceactor', 'http://www.wikidata.org/entity/Q1264306'), ('voiceactorname', 'Pat Carroll')]
[('film', 'http://www.wikidata.org/entity/Q869993'), ('name', 'A Goofy Movie'), ('voiceactor', 'http://www.wikidata.org/entity/Q311987'), ('voiceactorname', 'Rob Paulsen')]
[('film', 'http://www.wikidata.org/entity/Q869993'), ('name', 'A Goofy Movie'), ('voiceactor', 'http://www.wikidata.org/e

In [41]:
# aim: find the number of films produced by the 'Walt Disney Animation Studios' with their corresponding 'voice actor'(s)
# result: we found the number of films produced by the company 'Walt Disney Animation Studios' with their 'voice actor' (s)

queryString = """
SELECT DISTINCT ?voiceactor ?voiceactorname (COUNT(?film) AS ?numberfilms)WHERE{
    {
        SELECT DISTINCT ?voiceactor ?voiceactorname ?film WHERE{
            ?film wdt:P725 ?voiceactor .
            ?voiceactor sc:name ?voiceactorname .
            {
                SELECT DISTINCT ?film WHERE{
                    ?film wdt:P179 wd:Q56070713 .
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film WHERE {
                    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
                    ?film wdt:P272 wd:Q1047410 ;
                        # get the type of film
                        wdt:P31 ?instanceof .
                    # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
                    # and 'Walt Disney Animation Studios film'
                    FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        }
    }
} GROUP BY ?voiceactor ?voiceactorname
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('voiceactor', 'http://www.wikidata.org/entity/Q3453634'), ('voiceactorname', 'Ruth Pointer'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q613378'), ('voiceactorname', 'Bruno Campos'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q376981'), ('voiceactorname', 'Peter Firth'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q526620'), ('voiceactorname', 'Alfre Woodard'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q6307858'), ('voiceactorname', 'Julianne Buescher'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q272960'), ('voiceactorname', 'Stefanie Scott'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/entity/Q313266'), ('voiceactorname', 'John DiMaggio'), ('numberfilms', '3')]
[('voiceactor', 'http://www.wikidata.org/entity/Q230383'), ('voiceactorname', 'Kelly Macdonald'), ('numberfilms', '1')]
[('voiceactor', 'http://www.wikidata.org/en

In [42]:
# aim: find the number of films produced by the 'Walt Disney Animation Studios' with their corresponding 'voice actor'(s)
# result: we found the number of films produced by the company 'Walt Disney Animation Studios' with their 'voice actor' (s)

queryString = """
SELECT DISTINCT ?voiceactor ?voiceactorname (COUNT(?film) AS ?numberfilms)WHERE{
    {
        SELECT DISTINCT ?voiceactor ?voiceactorname ?film WHERE{
            ?film wdt:P725 ?voiceactor .
            ?voiceactor sc:name ?voiceactorname .
            {
                SELECT DISTINCT ?film WHERE{
                    ?film wdt:P179 wd:Q56070713 .
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film WHERE {
                    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
                    ?film wdt:P272 wd:Q1047410 ;
                        # get the type of film
                        wdt:P31 ?instanceof .
                    # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
                    # and 'Walt Disney Animation Studios film'
                    FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        }
    }
} GROUP BY ?voiceactor ?voiceactorname
HAVING (COUNT(?film)>5)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('voiceactor', 'http://www.wikidata.org/entity/Q296577'), ('voiceactorname', 'Frank Welker'), ('numberfilms', '14')]
[('voiceactor', 'http://www.wikidata.org/entity/Q328456'), ('voiceactorname', 'Jennifer Darling'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q2451247'), ('voiceactorname', 'Pat Buttram'), ('numberfilms', '6')]
[('voiceactor', 'http://www.wikidata.org/entity/Q349350'), ('voiceactorname', 'Alan Tudyk'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q3603'), ('voiceactorname', 'Jack Angel'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q24574600'), ('voiceactorname', 'Terri Douglas'), ('numberfilms', '6')]
[('voiceactor', 'http://www.wikidata.org/entity/Q2739967'), ('voiceactorname', 'Jimmy MacDonald'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q310292'), ('voiceactorname', 'Jim Cummings'), ('numberfilms', '16')]
[('voiceactor', 'http://www.wikidata.org/entity/Q

Final query for this task

In [43]:
# aim: find the 'voice actor'(s) who acted in more than 5 films produced by the company 'Walt Disney Animation Studios'
# result: we found the 'voice actor'(s) who acted in more than 5 films produced by the company 'Walt Disney Animation Studios'

queryString = """
SELECT DISTINCT ?voiceactor ?voiceactorname (COUNT(?film) AS ?numberfilms)WHERE{
    {
        SELECT DISTINCT ?voiceactor ?voiceactorname ?film WHERE{
            ?film wdt:P725 ?voiceactor .
            ?voiceactor sc:name ?voiceactorname .
            {
                SELECT DISTINCT ?film WHERE{
                    ?film wdt:P179 wd:Q56070713 .
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film WHERE {
                    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
                    ?film wdt:P272 wd:Q1047410 ;
                        # get the type of film
                        wdt:P31 ?instanceof .
                    # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
                    # and 'Walt Disney Animation Studios film'
                    FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        }
    }
} GROUP BY ?voiceactor ?voiceactorname
HAVING (COUNT(?film)>5)
"""

print("Results")
x=run_query(queryString)

Results
[('voiceactor', 'http://www.wikidata.org/entity/Q296577'), ('voiceactorname', 'Frank Welker'), ('numberfilms', '14')]
[('voiceactor', 'http://www.wikidata.org/entity/Q328456'), ('voiceactorname', 'Jennifer Darling'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q2451247'), ('voiceactorname', 'Pat Buttram'), ('numberfilms', '6')]
[('voiceactor', 'http://www.wikidata.org/entity/Q349350'), ('voiceactorname', 'Alan Tudyk'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q3603'), ('voiceactorname', 'Jack Angel'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q24574600'), ('voiceactorname', 'Terri Douglas'), ('numberfilms', '6')]
[('voiceactor', 'http://www.wikidata.org/entity/Q2739967'), ('voiceactorname', 'Jimmy MacDonald'), ('numberfilms', '7')]
[('voiceactor', 'http://www.wikidata.org/entity/Q310292'), ('voiceactorname', 'Jim Cummings'), ('numberfilms', '16')]
[('voiceactor', 'http://www.wikidata.org/entity/Q

## Task 6
Identify the BGP for the film "The Little Mermaid"

In [ ]:
# We already have the identifier for the film 'The Little Mermaid' from the above queries which is Q944287 so we proceed directly to a simple query that finds only the "The Little Mermaid" by 
# filtering with its id (this is done for the person who is correcting this notebook so that she/he is not required to go and check all the other query results)

In [12]:
# aim: get the bgp for 'The Little Mermaid'
# result: the bgp for 'The Little Mermaid'
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p ?o .
   # get the label
   ?s sc:name ?name .
   FILTER (?s=wd:Q944287)
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q944287'), ('name', 'The Little Mermaid')]
1


Final query for this task

In [3]:
# As for the error in task 4, we redo also this task by using the correct bgp that was found in task 4 that is Q190135
# aim: get the bgp for 'The Little Mermaid'
# result: the bgp for 'The Little Mermaid'
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   # bind something
   ?s ?p ?o .
   # get the label
   ?s sc:name ?name .
   FILTER (?s=wd:Q190135)
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q190135'), ('name', 'The Little Mermaid')]
1


## Task 7
Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label).

In [13]:
# aim: get all the properties of 'The Little Mermaid'
# result: all the properties of 'The Little Mermaid'

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q944287 ?p ?o .
   # get the label
   ?p sc:name ?name .
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2364'), ('name', 'production code')]
[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
[('p', 'http://www.wikidata.org/prop/direct/P2603'), ('name', 'Kinopoisk film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2

In [15]:
# aim: get all the people involved in the film 'The Little Mermaid' ('cast member', 'creator' and 'voice actor'). 
# result: the people involved in the film 'The Little Mermaid'

queryString = """
SELECT DISTINCT ?person ?personname
WHERE {
   # bind 'The Little Mermaid' to the persons who worked on it
   wd:Q944287 ?p ?person .
   # get the label
   ?person sc:name ?personname.
   FILTER (?p IN (wdt:P161, wdt:P170, wdt:P725))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q3565642'), ('personname', 'Walt Disney Television')]
[('person', 'http://www.wikidata.org/entity/Q3015230'), ('personname', 'Danny Cooksey')]
[('person', 'http://www.wikidata.org/entity/Q1138602'), ('personname', 'Jodi Benson')]
[('person', 'http://www.wikidata.org/entity/Q1264306'), ('personname', 'Pat Carroll')]
[('person', 'http://www.wikidata.org/entity/Q926362'), ('personname', 'Samuel E. Wright')]
[('person', 'http://www.wikidata.org/entity/Q5335683'), ('personname', 'Edan Gross')]
[('person', 'http://www.wikidata.org/entity/Q847124'), ('personname', 'Maurice LaMarche')]
[('person', 'http://www.wikidata.org/entity/Q1356690'), ('personname', 'Kenneth Mars')]
[('person', 'http://www.wikidata.org/entity/Q310292'), ('personname', 'Jim Cummings')]
[('person', 'http://www.wikidata.org/entity/Q304701'), ('personname', 'Bradley Pierce')]
10


In [16]:
# aim: get all the people involved in the film 'The Lion King' ('film crew member', 'film editor', 'performer', 
#     'producer', 'director', 'screenwriter', 'composer', 'production designer', 'executive producer', 'art director', 
#     'voice actor', 'animator', 'storyboard artist'). 
# result: the people involved in the film 'The Lion King'
queryString = """
SELECT DISTINCT ?person ?personname
WHERE {
   # bind 'The Lion King' to the persons who worked on it
   wd:Q36479 ?p ?person .
   # get the label
   ?person sc:name ?personname.
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q2396556'), ('personname', 'Mark Henn')]
[('person', 'http://www.wikidata.org/entity/Q16204825'), ('personname', 'Barry Kooser')]
[('person', 'http://www.wikidata.org/entity/Q19822599'), ('personname', 'Robert Walker')]
[('person', 'http://www.wikidata.org/entity/Q22115062'), ('personname', 'Aaron Blaise')]
[('person', 'http://www.wikidata.org/entity/Q27865787'), ('personname', 'Broose Johnson')]
[('person', 'http://www.wikidata.org/entity/Q27867881'), ('personname', 'Alex Kupershmidt')]
[('person', 'http://www.wikidata.org/entity/Q27913315'), ('personname', 'James Young Jackson')]
[('person', 'http://www.wikidata.org/entity/Q28790229'), ('personname', 'Phillip Young')]
[('person', 'http://www.wikidata.org/entity/Q28803503'), ('personname', 'Robert Stanton')]
[('person', 'http://www.wikidata.org/entity/Q28813470'), ('personname', 'Vera Pacheco')]
[('person', 'http://www.wikidata.org/entity/Q28842910'), ('personname', 'Lorna Cook')]
[(

In [17]:
# aim: get all the people involved in the film 'The Lion King' or 'The Little Mermaid' with the number 
#      of film they've been involved
# result: the people involved in the film 'The Lion King' or 'The Little Mermaid' with the number of film they've been involved.

queryString = """
SELECT DISTINCT ?person ?personname (COUNT(?film) AS ?filmcount)
WHERE {
   # bind 'The Lion King' or 'The Little Mermaid' to the persons who worked on it
   ?film ?p ?person .
   # get the label
   ?person sc:name ?personname .
   
   FILTER (?film IN (wd:Q36479,wd:Q944287)) .
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483,wdt:P161))
} GROUP BY ?person ?personname
ORDER BY DESC (?filmcount)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q310292'), ('personname', 'Jim Cummings'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q3531686'), ('personname', 'Tony Bancroft'), ('filmcount', '1')]
[('person', 'http://www.wikidata.org/entity/Q28813488'), ('personname', 'Lou Dellarosa'), ('filmcount', '1')]
[('person', 'http://www.wikidata.org/entity/Q259737'), ('personname', 'Madge Sinclair'), ('filmcount', '1')]
[('person', 'http://www.wikidata.org/entity/Q28858393'), ('personname', 'Raul Garcia'), ('filmcount', '1')]
[('person', 'http://www.wikidata.org/entity/Q3163012'), ('personname', 'Jason Weaver'), ('filmcount', '1')]
[('person', 'http://www.wikidata.org/entity/Q64046'), ('personname', 'Andreas Deja'), ('filmcount', '1')]
[('person', 'http://www.wikidata.org/entity/Q27600232'), ('personname', 'Chris Wahl'), ('filmcount', '1')]
[('person', 'http://www.wikidata.org/entity/Q10307886'), ('personname', 'John Ripa'), ('filmcount', '1')]
[('person', 'http://ww

In [18]:
# aim: get all the people involved in the film 'The Lion King' and 'The Little Mermaid' 
# result: the only person involved in the 2 films

queryString = """
SELECT DISTINCT ?person ?personname (COUNT(?film) AS ?filmcount)
WHERE {
   # bind 'The Lion King' or 'The Little Mermaid' to the persons who worked on it
   ?film ?p ?person .
   # get the label
   ?person sc:name ?personname .
   
   FILTER (?film IN (wd:Q36479,wd:Q944287)) .
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483,wdt:161))
} GROUP BY ?person ?personname
HAVING (COUNT(?film)>1)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q310292'), ('personname', 'Jim Cummings'), ('filmcount', '2')]
1


In [4]:
# We redo this task because of the same previous error
# aim: get all the people involved in the film 'The Little Mermaid' ('film crew member', 'film editor', 'performer', 
#     'producer', 'director', 'screenwriter', 'composer', 'production designer', 'executive producer', 'art director', 
#     'voice actor', 'animator', 'storyboard artist'). 
# result: the people involved in the film 'The Little Mermaid'

queryString = """
SELECT DISTINCT ?person ?personname
WHERE {
   # bind 'The Little Mermaid' to the persons who worked on it
   wd:Q190135 ?p ?person .
   # get the label
   ?person sc:name ?personname.
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1341627'), ('personname', 'Rod McKuen')]
[('person', 'http://www.wikidata.org/entity/Q2396556'), ('personname', 'Mark Henn')]
[('person', 'http://www.wikidata.org/entity/Q11853913'), ('personname', 'Barry Cook')]
[('person', 'http://www.wikidata.org/entity/Q12320808'), ('personname', 'Jorgen Klubien')]
[('person', 'http://www.wikidata.org/entity/Q28790229'), ('personname', 'Phillip Young')]
[('person', 'http://www.wikidata.org/entity/Q28803503'), ('personname', 'Robert Stanton')]
[('person', 'http://www.wikidata.org/entity/Q28813470'), ('personname', 'Vera Pacheco')]
[('person', 'http://www.wikidata.org/entity/Q3498870'), ('personname', 'Steve Bulen')]
[('person', 'http://www.wikidata.org/entity/Q7375942'), ('personname', 'Ruben A. Aquino')]
[('person', 'http://www.wikidata.org/entity/Q369276'), ('personname', 'Doug Lefler')]
[('person', 'http://www.wikidata.org/entity/Q1497666'), ('personname', 'Geefwee Boedoe')]
[('person', 'http:/

In [5]:
# aim: get all the people involved in the film 'The Lion King' or 'The Little Mermaid' with the number 
#      of film they've been involved
# result: the people involved in the film 'The Lion King' or 'The Little Mermaid' with the number of film they've been involved.

queryString = """
SELECT DISTINCT ?person ?personname (COUNT(?film) AS ?filmcount)
WHERE {
   # bind 'The Lion King' or 'The Little Mermaid' to the persons who worked on it
   ?film ?p ?person .
   # get the label
   ?person sc:name ?personname .
   
   FILTER (?film IN (wd:Q36479,wd:Q190135)) .
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483))
} GROUP BY ?person ?personname
ORDER BY DESC (?filmcount)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q3993230'), ('personname', 'Tony Fucile'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5228386'), ('personname', 'Dave Bossert'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q27600232'), ('personname', 'Chris Wahl'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5365068'), ('personname', 'Ellen Woodbury'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q18610100'), ('personname', 'Dorse Lanpher'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q27867909'), ('personname', 'Barry Temple'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5238736'), ('personname', 'David Pruiksma'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q64046'), ('personname', 'Andreas Deja'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q4772378'), ('personname', 'Anthony DeRosa'), ('filmcount', '2')]
[('person', 'ht

Final query for this task

In [6]:
# aim: get all the people involved in the film 'The Lion King' and 'The Little Mermaid' 
# result: the only person involved in the 2 films

queryString = """
SELECT DISTINCT ?person ?personname (COUNT(?film) AS ?filmcount)
WHERE {
   # bind 'The Lion King' or 'The Little Mermaid' to the persons who worked on it
   ?film ?p ?person .
   # get the label
   ?person sc:name ?personname .
   
   FILTER (?film IN (wd:Q36479,wd:Q190135)) .
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483))
} GROUP BY ?person ?personname
HAVING (COUNT(?film)>1)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q3993230'), ('personname', 'Tony Fucile'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5228386'), ('personname', 'Dave Bossert'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q27600232'), ('personname', 'Chris Wahl'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5365068'), ('personname', 'Ellen Woodbury'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q18610100'), ('personname', 'Dorse Lanpher'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q27867909'), ('personname', 'Barry Temple'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5238736'), ('personname', 'David Pruiksma'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q64046'), ('personname', 'Andreas Deja'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q4772378'), ('personname', 'Anthony DeRosa'), ('filmcount', '2')]
[('person', 'ht

## Task 8
Identify the BGP for Academy Awards

In [20]:
# aim: check the property 'nominated for' of 'The Lion King' to see if it is correlated to the 'Academy Awards'  
# result: we found a possible relation between that property and 'Academy Awards'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36479 wdt:P1411 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q488651'), ('name', 'Academy Award for Best Original Score')]
1


In [22]:
# aim: check if we find the property 'instance of' for 'Academy Award for Best Original Score'
# result: we found the property 'instance of' that might relate it to 'Academy Awards'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q488651 ?p ?obj .
   
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7084'), ('name', 'related category')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('name', 'official website')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/

Final query for this task

In [23]:
# aim: check the property 'instance of' for 'Academy Award for Best Original Score'
# result: we found that it relates to 'Academy Awards'
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q488651 wdt:P31 ?obj .
   
   # get the label
   ?obj sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q96474685'), ('name', 'award for best original music')]
2


## Task 9
For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

In [10]:
# aim: get all the properties of 'Academy Awards'
# result: all the properties of 'Academy Awards'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q19020 ?p ?obj .
   
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1407'), ('name', 'MusicBrainz series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003

In [19]:
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their 'nominated for'(s) filtering with with a regex to find only nominations containing the word 'academy' 
#      (case insensitive)
# result: we found the films produced by the company 'Walt Disney Animation Studios' with their 'nominated for'(s)

queryString = """
SELECT DISTINCT ?film ?filmname ?nominatedfor ?nominatedforname
WHERE {
    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
    ?film wdt:P272 wd:Q1047410 ;
           # get the label of the film
           sc:name ?filmname ;
           # get the 'voice actor'
           wdt:P1411 ?nominatedfor .
           
    ?nominatedfor sc:name ?nominatedforname.
    FILTER regex(?nominatedforname, "academy", "i")
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('filmname', 'Moana'), ('nominatedfor', 'http://www.wikidata.org/entity/Q106800'), ('nominatedforname', 'Academy Award for Best Animated Feature')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('filmname', 'Beauty and the Beast'), ('nominatedfor', 'http://www.wikidata.org/entity/Q488651'), ('nominatedforname', 'Academy Award for Best Original Score')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('filmname', 'Beauty and the Beast'), ('nominatedfor', 'http://www.wikidata.org/entity/Q102427'), ('nominatedforname', 'Academy Award for Best Picture')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('filmname', 'Beauty and the Beast'), ('nominatedfor', 'http://www.wikidata.org/entity/Q830079'), ('nominatedforname', 'Academy Award for Best Sound')]
[('film', 'http://www.wikidata.org/entity/Q379873'), ('filmname', 'Who Framed Roger Rabbit'), ('nominatedfor', 'http://www.wikidata.org/entity/Q393686'), ('nominat

In [20]:
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their number of nominations
# result: we found the films produced by the company 'Walt Disney Animation Studios' with their number of nominations

queryString = """
SELECT DISTINCT ?film ?filmname (COUNT(?nominatedfor) AS ?numbernominated)
WHERE {
    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its film
    ?film wdt:P272 wd:Q1047410 ;
           # get the label of the film
           sc:name ?filmname ;
           # get the 'voice actor'
           wdt:P1411 ?nominatedfor .
           
    ?nominatedfor sc:name ?nominatedforname.
    FILTER regex(?nominatedforname, "academy", "i")
}GROUP BY ?film ?filmname
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('filmname', 'Moana'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q2399982'), ('filmname', 'Tea for Two Hundred'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q1633009'), ('filmname', 'The Little Matchgirl'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q3259450'), ('filmname', 'Lorenzo'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q842306'), ('filmname', 'Saludos Amigos'), ('numbernominated', '2')]
[('film', 'http://www.wikidata.org/entity/Q16857388'), ('filmname', 'Feast'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q1440760'), ('filmname', 'Runaway Brain'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q215617'), ('filmname', 'Sleeping Beauty'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q246283'), ('filmname', 'Frozen'), ('numbernominated', '1')]
[('film

In [8]:
# As before, we redo also this task according to the new fixes
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their 'nominated for'(s) filtering with with a regex to find only nominations containing the word 'academy' 
#      (case insensitive)
# result: we found the films produced by the company 'Walt Disney Animation Studios' with their 'nominated for'(s)

queryString = """
SELECT DISTINCT ?film ?name ?nominatedfor ?nominatedforname WHERE{

    ?film wdt:P1411 ?nominatedfor .
       ?nominatedfor sc:name ?nominatedforname.
       FILTER regex(?nominatedforname, "academy", "i")

   {
       
   
        SELECT DISTINCT ?film ?name WHERE {
            {
                SELECT DISTINCT ?film ?name WHERE{
                    ?film wdt:P179 wd:Q56070713 ;
                       sc:name ?name.
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film ?name WHERE {
                    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
                    ?film wdt:P272 wd:Q1047410 ;
                        # get the label
                        sc:name ?name ;
                        # get the type of film
                        wdt:P31 ?instanceof .
                    # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
                    # and 'Walt Disney Animation Studios film'
                    FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        }
        
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q537407'), ('name', 'Mulan'), ('nominatedfor', 'http://www.wikidata.org/entity/Q22235305'), ('nominatedforname', 'Academy Award for Best Original Musical or Comedy Score')]
[('film', 'http://www.wikidata.org/entity/Q213787'), ('name', 'The Hunchback of Notre Dame'), ('nominatedfor', 'http://www.wikidata.org/entity/Q22235305'), ('nominatedforname', 'Academy Award for Best Original Musical or Comedy Score')]
[('film', 'http://www.wikidata.org/entity/Q218894'), ('name', 'Pocahontas'), ('nominatedfor', 'http://www.wikidata.org/entity/Q22235305'), ('nominatedforname', 'Academy Award for Best Original Musical or Comedy Score')]
[('film', 'http://www.wikidata.org/entity/Q134430'), ('name', 'Snow White and the Seven Dwarfs'), ('nominatedfor', 'http://www.wikidata.org/entity/Q22752811'), ('nominatedforname', 'Academy Award for Best Score, Adaptation or Treatment')]
[('film', 'http://www.wikidata.org/entity/Q204662'), ('name', 'The Sword in the S

Final query for this task

In [10]:
# aim: find all the films produced by the 'Walt Disney Animation Studios' with their number of nominations
# result: we found the films produced by the company 'Walt Disney Animation Studios' with their number of nominations

queryString = """
SELECT DISTINCT ?film ?name (COUNT(?nominatedfor) AS ?numbernominated) WHERE{

    ?film wdt:P1411 ?nominatedfor .
       ?nominatedfor sc:name ?nominatedforname.
       FILTER regex(?nominatedforname, "academy", "i")

   {
       
   
        SELECT DISTINCT ?film ?name WHERE {
            {
                SELECT DISTINCT ?film ?name WHERE{
                    ?film wdt:P179 wd:Q56070713 ;
                       sc:name ?name.
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film ?name WHERE {
                    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
                    ?film wdt:P272 wd:Q1047410 ;
                        # get the label
                        sc:name ?name ;
                        # get the type of film
                        wdt:P31 ?instanceof .
                    # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
                    # and 'Walt Disney Animation Studios film'
                    FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        }
        
   }
}GROUP BY ?film ?name
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q134430'), ('name', 'Snow White and the Seven Dwarfs'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q190135'), ('name', 'The Little Mermaid'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q215365'), ('name', 'Brother Bear'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q4341553'), ('name', 'Pinocchio'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q842306'), ('name', 'Saludos Amigos'), ('numbernominated', '2')]
[('film', 'http://www.wikidata.org/entity/Q36092'), ('name', 'Lilo & Stitch'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q189875'), ('name', 'Alice in Wonderland'), ('numbernominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q40895'), ('name', 'Dumbo'), ('numbernominated', '1')]
[('film', 'http://www.wikida